### Basic zip archive data access and usage
For simplicity, the following example loads only 200 sweeps directly from the zipfile into pandas DataFrame objects. These operations can be significantly accelerated using dask instead (see `dask_processing.ipynb`).

In order to run this yourself, you'll need to download a zip file archive, and adjust `data_path` accordingly.

In [1]:
# this is a development environment hack!
# after pip install of sea_ingest, you'd just import sea_ingest
import __init__ as sea_ingest
from labbench import stopwatch

data_path = 'data/NIT-2022-12-13.zip'
with stopwatch('ziparchive read'):
    dfs = sea_ingest.read_seamf_zipfile(data_path, allow=200, tz="America/New_York")

 INFO   2023-04-04 09:41:01.833 • labbench: ziparchive read 3.057 s elapsed


### Returned dictionary structure
The data are returned as a dictionary of `pd.DataFrame`, named by data product or metadata type.

### DataFrame structure
The data products are arranged as tables.
* The trace axis (time elapsed, FFT bin frequency, etc.) is given by the `column` attribute
* The trace index (timestamp, RF center frequency, and any trace specificiations like the detector) are arranged as levels of a multilevel index.

Some advantages to arranging the table this way:
* All data values (below, `dfs['pfp'].values`) are the same kind of quantity, in this case dBm/10 MHz. (TODO: attach units with pint :))
    - This means that operations like `10**(dfs['pfp']/10)` do not apply to the index
* We can use _any_ of the indexing metadata fields to query subsets of the data quickly

In [2]:
dfs['pfp']

Frame time elapsed (s)                                                    0.000000  \
datetime                         frequency    capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 3.555000e+09 min               rms      -101.5625   
                                              max               rms      -100.3125   
                                              mean              rms      -100.8750   
                                              min               peak      -94.8125   
                                              max               peak      -89.0625   
...                                                                            ...   
2022-10-27 01:41:27.014000-04:00 3.695000e+09 max               rms      -100.1250   
                                              mean              rms      -100.6875   
                                              min               peak      -94.2500   
                                              max               peak      -90.4375   
                                              mean              peak      -92.4375   

Frame time elapsed (s)                                                    0.000018  \
datetime                         frequency    capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 3.555000e+09 min               rms      -101.5000   
                                              max               rms      -100.5000   
                                              mean              rms      -100.8750   
                                              min               peak      -94.3750   
                                              max               peak      -90.2500   
...                                                                            ...   
2022-10-27 01:41:27.014000-04:00 3.695000e+09 max               rms      -100.1250   
                                              mean              rms      -100.6875   
                                              min               peak      -94.5000   
                                              max               peak      -90.2500   
                                              mean              peak      -92.5625   

Frame time elapsed (s)                                                    0.000036  \
datetime                         frequency    capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 3.555000e+09 min               rms      -101.5000   
                                              max               rms      -100.0625   
                                              mean              rms      -100.8750   
                                              min               peak      -94.6250   
                                              max               peak      -89.8750   
...                                                                            ...   
2022-10-27 01:41:27.014000-04:00 3.695000e+09 max               rms      -100.2500   
                                              mean              rms      -100.6875   
                                              min               peak      -94.6875   
                                              max               peak      -89.8750   
                                              mean              peak      -92.5625   

Frame time elapsed (s)                                                    0.000054  \
datetime                         frequency    capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 3.555000e+09 min               rms      -101.5625   
                                              max               rms      -100.2500   
                                              mean              rms      -100.8750   
                                              min               peak      -94.5000   
                                              max               peak      -89.1875   
...                                      

### Quick indexing tutorial

You can access each index level using the index value. One way is with the `.loc` accessor, specifying `axis=0` to indicate that all slices are applied to the index (otherwise the 2nd field applies to columns). For example:

In [3]:
dfs['pfp'].loc(axis=0)[:'2022-10-26 20:55', 3.555e9, 'max', 'rms']

,,,Frame time elapsed (s),0.000000,0.000018,0.000036,0.000054,0.000071,0.000089,0.000107,0.000125,0.000143,0.000161,...,0.009821,0.009839,0.009857,0.009875,0.009893,0.009911,0.009929,0.009946,0.009964,0.009982
datetime,frequency,capture_statistic,detector,,,,,,,,,,,,,,,,,,,,,
2022-10-26 20:50:03.701000-04:00,3.555000e+09,max,rms,-100.3125,-100.5000,-100.0625,-100.2500,-100.3125,-100.2500,-100.3750,-100.2500,-100.3750,-100.250,...,-100.4375,-100.2500,-100.3125,-100.3750,-100.3125,-100.3125,-100.3125,-100.3125,-100.3125,-100.3750
2022-10-26 20:51:27.659000-04:00,3.555000e+09,max,rms,-78.7500,-77.6250,-76.9375,-78.5000,-77.5000,-79.6250,-85.6875,-86.8125,-88.1875,-88.250,...,-85.8125,-88.2500,-88.0000,-88.0625,-86.8750,-83.0625,-82.2500,-78.5625,-77.6250,-78.2500
2022-10-26 20:52:51.837000-04:00,3.555000e+09,max,rms,-100.1875,-83.6875,-82.8125,-81.3125,-79.2500,-77.3125,-79.0625,-78.8750,-78.6250,-77.125,...,-100.3125,-100.3750,-100.0625,-100.2500,-100.2500,-100.3125,-100.3125,-100.1250,-100.2500,-100.3750
2022-10-26 20:54:16.029000-04:00,3.555000e+09,max,rms,-80.8125,-78.9375,-79.3750,-78.2500,-79.4375,-82.9375,-82.6875,-83.0000,-82.5000,-83.125,...,-77.9375,-78.4375,-80.8125,-84.0000,-86.6250,-87.5625,-83.9375,-82.1250,-82.6250,-81.3125


In many cases, we'd like to index a single value in a given level, especially for categorical data like the string-referred `capture_statistic` and `detector` fields. For this, pandas provides the `.xs` accessor.

In [4]:
dfs['pfp'].xs(key=3.555e9, level='frequency')

Frame time elapsed (s)                                       0.000000  \
datetime                         capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 min               rms      -101.5625   
                                 max               rms      -100.3125   
                                 mean              rms      -100.8750   
                                 min               peak      -94.8125   
                                 max               peak      -89.0625   
...                                                               ...   
2022-10-27 01:40:14.976000-04:00 max               rms       -77.9375   
                                 mean              rms       -83.7500   
                                 min               peak      -80.0625   
                                 max               peak      -71.8750   
                                 mean              peak      -76.0625   

Frame time elapsed (s)                                       0.000018  \
datetime                         capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 min               rms      -101.5000   
                                 max               rms      -100.5000   
                                 mean              rms      -100.8750   
                                 min               peak      -94.3750   
                                 max               peak      -90.2500   
...                                                               ...   
2022-10-27 01:40:14.976000-04:00 max               rms       -77.8125   
                                 mean              rms       -83.9375   
                                 min               peak      -79.8750   
                                 max               peak      -70.8750   
                                 mean              peak      -76.1875   

Frame time elapsed (s)                                       0.000036  \
datetime                         capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 min               rms      -101.5000   
                                 max               rms      -100.0625   
                                 mean              rms      -100.8750   
                                 min               peak      -94.6250   
                                 max               peak      -89.8750   
...                                                               ...   
2022-10-27 01:40:14.976000-04:00 max               rms       -78.9375   
                                 mean              rms       -84.1875   
                                 min               peak      -79.8125   
                                 max               peak      -72.3750   
                                 mean              peak      -76.1875   

Frame time elapsed (s)                                       0.000054  \
datetime                         capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 min               rms      -101.5625   
                                 max               rms      -100.2500   
                                 mean              rms      -100.8750   
                                 min               peak      -94.5000   
                                 max               peak      -89.1875   
...                                                               ...   
2022-10-27 01:40:14.976000-04:00 max               rms       -79.5000   
                                 mean              rms       -86.1250   
                                 min               peak      -80.8750   
                                 max               peak      -73.5000   
                                 mean              peak      -77.1250   

Frame time elapsed (s)                                       0.000071  \
datetime                         capture_statistic detector             
2022-10-26 20:50:03.701000-04:00 min               rms

For flexible queries to single index values of a specified data product and multiple index levels at a time, you can also use `seamf.trace`. In the following example, notice that this drops the selected levels from the index. 

In [5]:
sea_ingest.trace(dfs, 'pfp', frequency=3.555e9, capture_statistic='max', detector='rms').loc[:'2022-10-26 20:55']

Frame time elapsed (s),0.000000,0.000018,0.000036,0.000054,0.000071,0.000089,0.000107,0.000125,0.000143,0.000161,...,0.009821,0.009839,0.009857,0.009875,0.009893,0.009911,0.009929,0.009946,0.009964,0.009982
datetime,,,,,,,,,,,,,,,,,,,,,
2022-10-26 20:50:03.701000-04:00,-100.3125,-100.5000,-100.0625,-100.2500,-100.3125,-100.2500,-100.3750,-100.2500,-100.3750,-100.250,...,-100.4375,-100.2500,-100.3125,-100.3750,-100.3125,-100.3125,-100.3125,-100.3125,-100.3125,-100.3750
2022-10-26 20:51:27.659000-04:00,-78.7500,-77.6250,-76.9375,-78.5000,-77.5000,-79.6250,-85.6875,-86.8125,-88.1875,-88.250,...,-85.8125,-88.2500,-88.0000,-88.0625,-86.8750,-83.0625,-82.2500,-78.5625,-77.6250,-78.2500
2022-10-26 20:52:51.837000-04:00,-100.1875,-83.6875,-82.8125,-81.3125,-79.2500,-77.3125,-79.0625,-78.8750,-78.6250,-77.125,...,-100.3125,-100.3750,-100.0625,-100.2500,-100.2500,-100.3125,-100.3125,-100.1250,-100.2500,-100.3750
2022-10-26 20:54:16.029000-04:00,-80.8125,-78.9375,-79.3750,-78.2500,-79.4375,-82.9375,-82.6875,-83.0000,-82.5000,-83.125,...,-77.9375,-78.4375,-80.8125,-84.0000,-86.6250,-87.5625,-83.9375,-82.1250,-82.6250,-81.3125
